
# IRENE-2983

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf

In [ ]:
from invisible_cities.core.core_functions import define_window

In [ ]:
from invisible_cities.core.core_functions import lrange

In [ ]:
from collections import namedtuple

In [ ]:
from   invisible_cities.core.params import S12Params as S12P

In [ ]:
from   invisible_cities.core.params import SensorParams

In [ ]:
from invisible_cities.cities.irene import Irene, IRENE

## Preproc Steps

**Irene takes care of the following chores**

1. Deconvolute RWF
2. Compute calibrated sum of PMTs (including MAU threshold)
3. Zero Suppression (ZS) in PMTs
4. S1 search (analysis specific)
5. S2 search (analysis specific) and rebin
6. ZS in SiPMs
7. Select S2 in ZS SiPMs 

For fast pre-proc, all the above steps must be computed on the fly (e.g, minimal access to disk) and
using pre-compiled (cython) functions in calculation-intensive parts (loops)

## Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/2983/run_2983.gdcsnext.9600.next1el_2983.root.h5')
PMP_file = os.path.join(os.environ['IC_DATA'],'LSC/PMP_2983/run_2983.9600.PMP.h5')
CONF_file = os.path.join(os.environ['IC_CONFIG'], 'irene_Kr2983.conf')

In [ ]:
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(2983)
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### Instance

In [ ]:
s1par = S12P(tmin       = 10 * units.mus,
                 tmax   = 590 * units.mus,
                 stride = 4,
                 lmin   = 6,
                 lmax   = 16,
                 rebin  = False)

s2par = S12P(tmin       = 590 * units.mus,
                 tmax   = 620 * units.mus,
                 stride = 40,
                 lmin   = 100,
                 lmax   = 1000,
                 rebin  = True)

irene = Irene(run_number  = 2983,
                 files_in    = [RWF_file],
                 file_out    = PMP_file,
                 compression = 'ZLIB4',
                 nprint      = 1,
                 n_baseline  = 38000,
                 thr_trigger = 5.0 * units.adc,
                 n_MAU       = 100,
                 thr_MAU     = 3.0 * units.adc,
                 thr_csum_s1 = 0.2 * units.adc,
                 thr_csum_s2 = 1.0 * units.adc,
                 n_MAU_sipm  = 100,
                 thr_sipm    = 5.0 * units.pes,
                 s1_params   = s1par,
                 s2_params   = s2par,
                 thr_sipm_s2 = 30. * units.pes)

In [ ]:
def irene_run(irene, evt):
    # Check that PMAPS tables are written correctly
    
    with tb.open_file(irene.input_files[0], "r") as h5in:
        with tb.open_file(irene.output_file, "w",
            filters = tbl.filters(irene.compression)) as pmap_file:

            irene._set_pmap_store(pmap_file)
            irene.eventsInfo = h5in.root.Run.events
            pmtrwf  = h5in.root.RD. pmtrwf
            sipmrwf = h5in.root.RD.sipmrwf

            CWF = irene.deconv_pmt(pmtrwf[evt])
            csum, csum_mau = irene.calibrated_pmt_sum(CWF)
            s1_ene, s1_indx = irene.csum_zs(csum_mau,
                                             threshold = irene.thr_csum_s1)
            s2_ene, s2_indx = irene.csum_zs(csum,
                                             threshold = irene.thr_csum_s2)

            sipmzs = irene.calibrated_signal_sipm(sipmrwf[evt])
            S1, S2 = irene.find_S12(s1_ene, s1_indx, s2_ene, s2_indx)
            S2Si = irene.find_S2Si(S2, sipmzs)
            irene._store_pmaps(0, 0, S1, S2, S2Si)

            assert irene.s1t == pmap_file.root.PMAPS.S1
            assert irene.s2t == pmap_file.root.PMAPS.S2
            assert irene.s2sit == pmap_file.root.PMAPS.S2Si
            
    return CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si 



In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene, evt=0)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=False, window_size=2000)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=590
tmax=620
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
S1

In [ ]:
tmin=0
tmax=200
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=5)

In [ ]:
tmin=240
tmax=280
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=20)

In [ ]:
tmin=367
tmax=369
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=10)

In [ ]:
csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene, evt=100)

In [ ]:
S1

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
S2

### Step 1: Deconvolution (from RWF to CWF)

#### A deconvolution algorithm (V. Herrero, JJGC) takes the distorted RWF and produces corrected waveforms (CWF) where the effect of the FEE has been deconvoluted.

Found in module **invisible_cities.sierpe.blr**

Signature: **deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=28000, thr_trigger=5)**

**Run over one event**

In [ ]:
event = 0
BLR = pmtblr[event]
RWF = pmtrwf[event]

In [ ]:
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=28000, thr_trigger=5)
t1 = time()        
dt = t1 - t0
print("run  in {} s".format(dt))

To plot the waveforms use **invisible_cities.core.wfm_functions** function:

**plot_pmt_waveforms(WF)** where WF is a vector of waveforms

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=True, window_size=2000)

Notice the small S1, which is observed in a few of the waveforms. S1 for 40 keV electrons is small (a few PES) and thus the probability of observing zero PES in one of the PMTs is high. 

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/2983/run_2983.gdcsnext.9999.next1el_2983.root.h5')
PMP_file = os.path.join(os.environ['IC_DATA'],'LSC/PMP_2983/run_2983.gdcsnext.9999.next1el_2983.root.h5')

In [ ]:
if h5rwf:
    h5rwf.close()
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

In [ ]:
event = 0
BLR = pmtblr[event]
RWF = pmtrwf[event]

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=True, window_size=2000)

#### Plot BLR on top of CWF. Excellent match

In [ ]:
wfm.plot_wfa_wfb(CWF, BLR, zoom=True, window_size=200) 

### Step 2: calibrated PMT sum

#### The PMTs of the energy plane are calibrated (dividing by calibration constants) and added. 

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr)
csum, csum_mau = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
tmin=120
tmax=130
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)
#mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)

#### Notice that MAU introduces a bias in the signal region, thus csum in the signal region must be used without MAU

In [ ]:
tmin=99
tmax=101
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

#### MAU works very nicely to search for S1, since it eliminates the pedestal. 

#### Test: the calibrated sum of the BLR and the CWF must be very close

In [ ]:
def calibrated_sum_cwf_blr_diff(pmtrwf, pmtblr, event_list=[10], 
                                plot=False, window=200):
    """Test that the calibrated sum of the BLR and the CWF is the same within tolerance"""
    
    DataPMT = load_db.DataPMT()
    coeff_c = abs(DataPMT.coeff_c.values)
    coeff_blr = abs(DataPMT.coeff_blr.values)
    adc_to_pes = abs(DataPMT.adc_to_pes.values)
    DIFF = []
    for event in event_list:     
        CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
        csum_cwf, _ = cpf.calibrated_pmt_sum(CWF, adc_to_pes)
        csum_blr, _ = cpf.calibrated_pmt_sum(pmtblr[event].astype(np.float64), adc_to_pes)
        diff = csum_cwf - csum_blr
        norm = np.sum(csum_blr)
        if plot:
            wfm.plot_waveform(diff, zoom=plot, window_size=window)
            plt.show()
        DIFF.append(np.sum(diff)/norm)
    return DIFF


In [ ]:
diff = calibrated_sum_cwf_blr_diff(pmtrwf, pmtblr, event_list=lrange(10))

In [ ]:
mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

### Step 3: Zero suppression

#### ZS is performed separately for S1 (using csum_MAU) and for S2 (using csum)

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum, csum_mau = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
s2_ene, s2_indx = cpf.wfzs(csum, threshold=1.0*units.pes)
s2_t = cpf.time_from_index(s2_indx)
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.2*units.pes)
s1_t = cpf.time_from_index(s1_indx)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(s2_t/units.mus, s2_ene, title="calibrated sum, ZS",
                signal_start=120, signal_end=130, 
                ymax = 150, 
                t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(s1_t/units.mus, s1_ene, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 2, t_units='mus', units="pes")

### Step 4: Find S12 

**S1 and S2 signals are found from the ZS waveform**

1. S1: Search the first 600 mus (in the MC S1 is always at 100 mus but not in the data). The S1 signal must be at least 100 ns long (this is the minimum length of S1 after the shaping of the electronics) and not larger than 500 ns. The stride counts the number of bins that need to have signal. A stride of 4 means that there must be at least 0.5 pes each 100 ns (4 x 25 ns, where 25 ns is the DAQ sampling time). 

2. Search in the whole window (in the MC data, S2 can be anywhere after 100 mus, in the data it will tipically be placed in the middle of the DAQ window, thus one would search in the range [600, 1200] ns. The minimum length is 120 bins (120 x 25 = 3000 ns), no limit in the max length of the signal is not restricted (in order to avoid a bias, but one could further select the signal with a max width cut), the stride is taken to 40 (1 mus) and the signal is rebinned also in bins of 1 mus

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum, csum_mau = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
s2_ene, s2_indx = cpf.wfzs(csum, threshold=1.0*units.pes)
s2_t = cpf.time_from_index(s2_indx)
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.2*units.pes)
s1_t = cpf.time_from_index(s1_indx)

S1 = cpf.find_S12(s1_ene, s1_indx, tmin=0,  tmax=110*units.mus, 
                 lmin=4, lmax=16, stride=4,
                 rebin=False)

S2 = cpf.find_S12(s2_ene, s2_indx, tmin=110*units.mus,  tmax=1100*units.mus, 
                 lmin=80, lmax=10000, stride=40,
                 rebin=True, rebin_stride=40)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

#### S1 and and S2 signals

In [ ]:
S1

In [ ]:
S2

There is one S2 in this event. The format of S2 is:

**{s2_number:[array(T), array(E)]}**

where the ZS waveform is expressed by the list **[array(T), array(E)]**

#### Structure of S1/S2
1. Dictionary index counts the number of peaks
2. Dict values are a list [T,E], where T and E are numpy arrays of Time and Energy

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

### SiPMs

In the case of MC Many of the SiPMs will contain exact zeros, since they had been ZS already at DIOMIRA level. For example, if we plot the first 16 SiPMs, we can see they are all exactly zero.

Find SiPMs with signal: the threshold is set to 1 adc count to simply get rid of all SiPMs which have exactly zero signal (the output of DIOMIRA is already ZS)

In [ ]:
event=1
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=1*units.adc) 

In [ ]:
sipm_i

In [ ]:
sf.plot_sipm_list(sipmrwf[event], sipm_i)

### Subtract baseline, and set a cut to supress dark current

in the MC the data comes already ZS. In data one needs to set a cut in order to ZS the SiPMs. 

In [ ]:
event=1
t0 = time()
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
t1 = time()        
dt = t1 - t0

print("run in {} s".format(dt))
print('number of SiPM with signal = {}'.format(len(SIPM)))

### PMAPS

#### Structure of a PMAP
1. S1 and S2 are dictionaries: each dictionary index is one S1/S2 candidate. Each dictionary value is a list which contains two elements. Element [0] is a np vector of times, element[1] is a np vector of energies in pes.
2. S2Si is a dictionary. Each index correspond to the S2 index. Each value is a list, whose length is equal to the number of SiPM with no zero signal in the S2 window. The list has as a first element the SiPM index and as a second the energy of each SiPM in the S2 window. Time is not neeeded (comes with S2).

#### S2 window
1. Given an S2 (T,E), obtain the relevant index range. 
2. Given a vector with SIPMs (energies above threshold), returns
    a list of np arrays. Each element of the list is the S2 window 
    in the SiPM (if not zero)
3. Given a vector with SIPMs (energies above threshold), and a 
    dictionary of S2s, S2d, returns a dictionary of SiPMs-S2.
    Each index of the dictionary correspond to one S2 and is
    a list of np arrays. Each element of the list is the S2 window 
    in the SiPM (if not zero)

## The full pre-proc 

In [ ]:
S12P= namedtuple('S12Params', 'tmin tmax lmin lmax stride')

In [ ]:
PMPT = namedtuple('PMPT', 'thr_s1 thr_s2 thr_MAU thr_sipm thr_SIPM')

In [ ]:
s1par = S12P(tmin=90*units.mus, tmax=110*units.mus, lmin=4, lmax=20, stride=4)

In [ ]:
s1par

In [ ]:
s2par = S12P(tmin=110*units.mus, tmax=1190*units.mus, lmin=80, lmax=200000, stride=40)

In [ ]:
s2par

In [ ]:
thr = PMPT(thr_s1=0.2*units.pes, 
                 thr_s2=1*units.pes, 
                 thr_MAU=3*units.adc, 
                 thr_sipm=5*units.pes, 
                 thr_SIPM=30*units.pes )

In [ ]:
thr

In [ ]:
Csum= namedtuple('CalibratedSum', 'csum csum_mau')

In [ ]:
Pmp = namedtuple('PMaps', 'S1 S2 S2Si')

In [ ]:
a = None

In [ ]:
if a:
    print('a is not monja')
else:
    print('a is monja')


In [ ]:
def compute_pmp(pmtrwf, s1par, s2par2, thresholds, event):
    """Compute S12 for event"""
    
    s1_params = s1par
    s2_params = s2par
    thr = thresholds
    
    # data base
    DataPMT = load_db.DataPMT()
    adc_to_pes = abs(DataPMT.adc_to_pes.values)
    coeff_c = abs(DataPMT.coeff_c.values)
    coeff_blr = abs(DataPMT.coeff_blr.values)
    DataSiPM = load_db.DataSiPM()
    adc_to_pes_sipm = DataSiPM.adc_to_pes.values

    # deconv
    CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
    
    # calibrated sum
    csum, csum_mau = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=thr.thr_MAU)
     
    # zs sum
    s2_ene, s2_indx = cpf.wfzs(csum, threshold=thr.thr_s2)
    s2_t = cpf.time_from_index(s2_indx)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=thr.thr_s1)
    s1_t = cpf.time_from_index(s1_indx)

    # S1 and S2
    S1 = cpf.find_S12(s1_ene, s1_indx, rebin = False, **s1_params._asdict())
    S2 = cpf.find_S12(s2_ene, s2_indx, rebin=True, **s2_params._asdict())
    
    #S2Si
    sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=thr.thr_sipm, n_MAU=100)
    SIPM = cpf.select_sipm(sipm)
    S2Si = pf.sipm_s2_dict(SIPM, S2, thr=thr.thr_SIPM)
    return Csum(csum=csum, csum_mau=csum_mau), Pmp(S1=S1, S2=S2, S2Si=S2Si)

In [ ]:
CSUM, PMP = compute_pmp(pmtrwf, s1par, s2par, thr, event=0)

In [ ]:
def plot_signal_vs_time_mus(signal,
                            t_min      =    0,
                            t_max      = 1200,
                            signal_min =    0,
                            signal_max =  200,
                            label=''):
    """Plot signal versus time in mus (tmin, tmax in mus). """
    tstep = 25 # in ns
    PMTWL = signal.shape[0]
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    ax1 = plt.subplot(1, 1, 1)
    ax1.set_xlim([t_min, t_max])
    ax1.set_ylim([signal_min, signal_max])
    mpl.set_plot_labels(xlabel = "t (mus)",
                    ylabel = "signal (pes/adc)")
    plt.plot(signal_t, signal, label=label)
    legend = plt.legend(loc='upper right')
    for label in legend.get_texts():
        label.set_fontsize('small')

In [ ]:
def plot_csum_in_mus(CSUM, tmin, tmax, signal_min, signal_max):
    """Plots in mus (notice units)"""
    plot_signal_vs_time_mus(CSUM.csum, 
                                t_min=tmin, t_max=tmax, 
                                signal_min=signal_min, signal_max=signal_max,
                                label='CSUM')
    plot_signal_vs_time_mus(CSUM.csum_mau, 
                                t_min=tmin, t_max=tmax, 
                                signal_min=signal_min, signal_max=signal_max,
                                label='CSUM_MAU')

In [ ]:
plot_csum_in_mus(CSUM, tmin=120, tmax=130, signal_min=-2, signal_max=120)

In [ ]:
plot_csum_in_mus(CSUM, tmin=99, tmax=101, signal_min=-2, signal_max=2)

In [ ]:
pf.scan_s12(PMP.S1)

In [ ]:
pf.scan_s12(PMP.S2)

In [ ]:
PMP.S2Si

#### Plot SiPMs

In [ ]:
pf.plot_s2si_map(PMP.S2Si)

#### event=1

In [ ]:
CsPl= namedtuple('CsumPlot', 'tmin tmax signal_min signal_max')

In [ ]:
def describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=0):
    CSUM, PMP = compute_pmp(pmtrwf, s1par, s2par, thr, event=event)
    plot_csum_in_mus(CSUM, **csumpl_s1._asdict())
    plt.show()
    plot_csum_in_mus(CSUM, **csumpl_s2._asdict())
    plt.show()
    pf.scan_s12(PMP.S1)
    plt.show()
    pf.scan_s12(PMP.S2)
    plt.show()
    pf.plot_s2si_map(PMP.S2Si)
    plt.show()

In [ ]:
csumpl_s2 = CsPl(tmin=120, tmax=130, signal_min=-2, signal_max=120)

In [ ]:
csumpl_s1 = CsPl(tmin=99, tmax=101, signal_min=-2, signal_max=2)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=0)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=1)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=2)

## Case 2: electrons of 511 keV

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_PMAP.h5'

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
csumpl_s1 = CsPl(tmin=99, tmax=101, signal_min=-2, signal_max=20)
csumpl_s2 = CsPl(tmin=300, tmax=400, signal_min=-2, signal_max=350)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=0)

## Case 3: electrons of 1250 keV

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/electrons_1250keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_1250keV_z250_PMAP.h5'

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
csumpl_s1 = CsPl(tmin=99, tmax=101, signal_min=-2, signal_max=40)
csumpl_s2 = CsPl(tmin=110, tmax=160, signal_min=-2, signal_max=900)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=0)

## Case 4: electrons of 2500 keV

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/electrons_2500keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_2500keV_z250_PMAP.h5'

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
csumpl_s1 = CsPl(tmin=99, tmax=101, signal_min=-2, signal_max=90)
csumpl_s2 = CsPl(tmin=90, tmax=300, signal_min=-2, signal_max=500)

In [ ]:
describe_event(pmtrwf, s1par, s2par, thr, csumpl_s1, csumpl_s2, event=2)

End of notebook